# Assignment 2 Supplement

Create a dataframe from the amazon meta data

In [1]:
import networkx as net 
import pandas as pd
import numpy as np
import os 
import csv

## Key

Manually Scanning the metadata text file and reviewing the attributes that are available 

- Id: Product ID number 

- ASIN: Amazon Standard Identification Number

- title: Product name 

- group: Product category 

- salesrank: Total sales ranking among the products in the data set 

- similar: ASIN Number of similar products found in the data set 

- categories: Number of unique categories
    - Sub category names exist but only number of unique categories was extracted 
    - (|Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Preaching[12368])
    

- reviews: Numbers of total, downloaded, and average review rating (total: 2  downloaded: 2  avg rating: 5)
    - Detailed metadata on the individual review exist that include but only the above was extracted 
        - Date
        - customer ID
        - rating
        - votes
        - number that found the review helpful 

In [4]:
file = '/Users/dirkhartog/Desktop/CUNY_MSDS/DATA_620/Assignment2/amazon-meta.txt'

with open(file, 'r') as f:
    id_pos_list = []
    content = f.readlines()
    for i, item in enumerate(content):
        #item_cleaned = item.strip('\n')
        if item.startswith('Id:'):
            id_pos_list.append(i)

In [6]:
len(id_pos_list)

548552

In [8]:
id_pos_list[0:4]

[3, 7, 20, 43]

In [10]:
# Extract the first Id
content[3:7]

['Id:   0\n', 'ASIN: 0771044445\n', '  discontinued product\n', '\n']

In [12]:
# Extract the second Id
content[7:20]

['Id:   1\n',
 'ASIN: 0827229534\n',
 '  title: Patterns of Preaching: A Sermon Sampler\n',
 '  group: Book\n',
 '  salesrank: 396585\n',
 '  similar: 5  0804215715  156101074X  0687023955  0687074231  082721619X\n',
 '  categories: 2\n',
 '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Preaching[12368]\n',
 '   |Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Sermons[12370]\n',
 '  reviews: total: 2  downloaded: 2  avg rating: 5\n',
 '    2000-7-28  cutomer: A2JW67OY8U6HHK  rating: 5  votes:  10  helpful:   9\n',
 '    2003-12-14  cutomer: A2VE83MZF98ITY  rating: 5  votes:   6  helpful:   5\n',
 '\n']

In [14]:
# Create new list of lists where each list contains the product information

content_list = []

pos = 1
for i in id_pos_list:
    if i != id_pos_list[-1]:
        content_list.append(content[i:id_pos_list[pos]])
        pos += 1 
    else: 
        content_list.append(content[i:])

In [16]:
# Confirm length of the content list 

len(content_list)

548552

In [18]:
# Copy content list 

content_copied = content_list.copy()

In [20]:
# Create function that updates a dictonary with date for each meta data category

def compare_attributes(d_update, d_get):
    
    """Compare two dictionaries and updating the values in the first dictionary with values in the second,
    adding in np.nan if not a key"""
    
    for k in d_update.keys():
        if k in d_get.keys():
            d_update[k].append(d_get[k])
        else:
            d_update[k].append(np.nan)

In [59]:
# Create dictionary to apply compare attributes to 

content_dict = {"ID":[],
                "ASIN":[],
                "TITLE":[],
                "GROUP":[],
                "SALESRANK":[],
                "SIMILAR":[],
                "CATEGORIES":[],
                "REVIEWS":[]
               }
    
for l in content_copied: # Loop through the list of products
    attr_dict = {}
    for attr in l: # Loop thorught the attributes in the product 
        attr_split = attr.lstrip().upper().split(":", 1)
        
        if len(attr_split) > 1:
            attr_dict[attr_split[0]] = attr_split[1].rstrip("\n")
            
    #Run a function that compares the attributes in the dictionary to what is in the attribute list
    compare_attributes(content_dict, attr_dict)
    #print(attr_dict)

# Create a pandas dataframe from the content dictionary
df = pd.DataFrame(content_dict)

In [74]:
# Create a copy of the data frame to begin tidy and cleaning columns


metadata_df = df.copy()

# View the first few rows
metadata_df.head()

,ID,ASIN,TITLE,GROUP,SALESRANK,SIMILAR,CATEGORIES,REVIEWS
0,0,0771044445,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0827229534,PATTERNS OF PREACHING: A SERMON SAMPLER,BOOK,396585,5 0804215715 156101074X 0687023955 068707...,2,TOTAL: 2 DOWNLOADED: 2 AVG RATING: 5
2,2,0738700797,CANDLEMAS: FEAST OF FLAMES,BOOK,168596,5 0738700827 1567184960 1567182836 073870...,2,TOTAL: 12 DOWNLOADED: 12 AVG RATING: 4.5
3,3,0486287785,WORLD WAR II ALLIED FIGHTER PLANES TRADING CARDS,BOOK,1270652,0,1,TOTAL: 1 DOWNLOADED: 1 AVG RATING: 5
4,4,0842328327,LIFE APPLICATION BIBLE COMMENTARY: 1 AND 2 TI...,BOOK,631289,5 0842328130 0830818138 0842330313 084232...,5,TOTAL: 1 DOWNLOADED: 1 AVG RATING: 4


In [ ]:
## Tidy and Clean up data frame 

In [78]:
# Expand the SIMILAR column to extract the number of similarities and each ASIN

metadata_df["NUM_SIMILAR"] = metadata_df["SIMILAR"].str.lstrip().str.split(" ", n = 1).str.get(0)

metadata_df["SIMILAR_PRODUCT_ASIN"] = metadata_df["SIMILAR"].str.lstrip().str.split(" ", n = 1).str.get(1)

metadata_df.head()

,ID,ASIN,TITLE,GROUP,SALESRANK,SIMILAR,CATEGORIES,REVIEWS,NUM_SIMILAR,SIMILAR_PRODUCT_ASIN
0,0,0771044445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0827229534,PATTERNS OF PREACHING: A SERMON SAMPLER,BOOK,396585,5 0804215715 156101074X 0687023955 068707...,2,TOTAL: 2 DOWNLOADED: 2 AVG RATING: 5,5,0804215715 156101074X 0687023955 068707423...
2,2,0738700797,CANDLEMAS: FEAST OF FLAMES,BOOK,168596,5 0738700827 1567184960 1567182836 073870...,2,TOTAL: 12 DOWNLOADED: 12 AVG RATING: 4.5,5,0738700827 1567184960 1567182836 073870052...
3,3,0486287785,WORLD WAR II ALLIED FIGHTER PLANES TRADING CARDS,BOOK,1270652,0,1,TOTAL: 1 DOWNLOADED: 1 AVG RATING: 5,0,NaN
4,4,0842328327,LIFE APPLICATION BIBLE COMMENTARY: 1 AND 2 TI...,BOOK,631289,5 0842328130 0830818138 0842330313 084232...,5,TOTAL: 1 DOWNLOADED: 1 AVG RATING: 4,5,0842328130 0830818138 0842330313 084232861...


In [92]:
# Investigate the day in the review columns 

for row in metadata_df["REVIEWS"][0:3]:
    if row is np.nan:
        continue
    else:
        print(row.index("  "))

9
10


In [104]:
# Expand the REVIEWS column to  Extract the total, downloaded, and avg rating and create 3 new columns 


def create_new_col(df, col_name = "REVIEWS", substr = "  ", index_pos):
    
    """Function to expand the REVIEWS column of the amazon metadata
    
    df: Current dataframe 
    col_name: Column name to apply th efunction; defautl = "REVIEWS"
    substr: String to split on; default = "  " (2 white spaces)
    index_pos: Position of information to grab in the inital split
    
    """
    review_info_split = df[col_name].str.split(substr).str.get(index_pos)
    review_info_list = []
    for row in review_info_split:
        if row is np.nan:
            review_info_list.append(np.nan)
        else:
            value = row.split(":")[1].strip(" ")
            review_info_list.append(value)
    
    return review_info_list

metadata_df["REVIEWS_TOTAL"] = create_new_col(metadata_df, "REVIEWS", "  ", 0)
metadata_df["REVIEWS_DOWNLOADED"] = create_new_col(metadata_df, "REVIEWS", "  ", 1)
metadata_df["REVIEWS_AVG_RATING"] = create_new_col(metadata_df, "REVIEWS", "  ", 2)

In [106]:
# Verify data 

metadata_df.head()

,ID,ASIN,TITLE,GROUP,SALESRANK,SIMILAR,CATEGORIES,REVIEWS,NUM_SIMILAR,SIMILAR_PRODUCT_ASIN,REVIEWS_TOTAL,REVIEWS_DOWNLOADED,REVIEWS_AVG_RATING
0,0,0771044445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0827229534,PATTERNS OF PREACHING: A SERMON SAMPLER,BOOK,396585,5 0804215715 156101074X 0687023955 068707...,2,TOTAL: 2 DOWNLOADED: 2 AVG RATING: 5,5,0804215715 156101074X 0687023955 068707423...,2,2,5
2,2,0738700797,CANDLEMAS: FEAST OF FLAMES,BOOK,168596,5 0738700827 1567184960 1567182836 073870...,2,TOTAL: 12 DOWNLOADED: 12 AVG RATING: 4.5,5,0738700827 1567184960 1567182836 073870052...,12,12,4.5
3,3,0486287785,WORLD WAR II ALLIED FIGHTER PLANES TRADING CARDS,BOOK,1270652,0,1,TOTAL: 1 DOWNLOADED: 1 AVG RATING: 5,0,NaN,1,1,5
4,4,0842328327,LIFE APPLICATION BIBLE COMMENTARY: 1 AND 2 TI...,BOOK,631289,5 0842328130 0830818138 0842330313 084232...,5,TOTAL: 1 DOWNLOADED: 1 AVG RATING: 4,5,0842328130 0830818138 0842330313 084232861...,1,1,4


In [122]:
# Inspect the unique values in the group column 

metadata_df["GROUP"].unique()

array([nan, ' BOOK', ' MUSIC', ' DVD', ' VIDEO', ' TOY', ' VIDEO GAMES',
       ' SOFTWARE', ' BABY PRODUCT', ' CE', ' SPORTS'], dtype=object)

In [128]:
# Clean up any white spaces left over in all columns

metadata_df = metadata_df.apply(lambda x: x.str.strip(), axis = 0)

In [132]:
# Verify data 
metadata_df["GROUP"].unique()

array([nan, 'BOOK', 'MUSIC', 'DVD', 'VIDEO', 'TOY', 'VIDEO GAMES',
       'SOFTWARE', 'BABY PRODUCT', 'CE', 'SPORTS'], dtype=object)

In [ ]:
# Convert data type in GROUP to category 
metadata_df["GROUP"] = metadata_df["GROUP"].astype("category")

In [134]:
# Get info about the data frame 

metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548552 entries, 0 to 548551
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   ID                    548552 non-null  object
 1   ASIN                  548552 non-null  object
 2   TITLE                 542684 non-null  object
 3   GROUP                 542684 non-null  object
 4   SALESRANK             542684 non-null  object
 5   SIMILAR               542684 non-null  object
 6   CATEGORIES            542684 non-null  object
 7   REVIEWS               542684 non-null  object
 8   NUM_SIMILAR           542684 non-null  object
 9   SIMILAR_PRODUCT_ASIN  379093 non-null  object
 10  REVIEWS_TOTAL         542684 non-null  object
 11  REVIEWS_DOWNLOADED    542684 non-null  object
 12  REVIEWS_AVG_RATING    542684 non-null  object
dtypes: object(13)
memory usage: 54.4+ MB


In [159]:
# Save original data frame

metadata_df.to_csv('/Users/dirkhartog/Desktop/CUNY_MSDS/DATA_620/Assignment2/amazon_meta.csv')

In [155]:
# Drop unneccessary columns to decrease data in the file 

metadata_df.drop(columns = ["SIMILAR", "REVIEWS"], inplace = True)

# Verify columns were dropped

metadata_df.head()

,ID,ASIN,TITLE,GROUP,SALESRANK,CATEGORIES,NUM_SIMILAR,SIMILAR_PRODUCT_ASIN,REVIEWS_TOTAL,REVIEWS_DOWNLOADED,REVIEWS_AVG_RATING
0,0,0771044445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0827229534,PATTERNS OF PREACHING: A SERMON SAMPLER,BOOK,396585,2,5,0804215715 156101074X 0687023955 0687074231...,2,2,5
2,2,0738700797,CANDLEMAS: FEAST OF FLAMES,BOOK,168596,2,5,0738700827 1567184960 1567182836 0738700525...,12,12,4.5
3,3,0486287785,WORLD WAR II ALLIED FIGHTER PLANES TRADING CARDS,BOOK,1270652,1,0,NaN,1,1,5
4,4,0842328327,LIFE APPLICATION BIBLE COMMENTARY: 1 AND 2 TIM...,BOOK,631289,5,5,0842328130 0830818138 0842330313 0842328610...,1,1,4


In [157]:
metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 548552 entries, 0 to 548551
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype   
---  ------                --------------   -----   
 0   ID                    548552 non-null  object  
 1   ASIN                  548552 non-null  object  
 2   TITLE                 542684 non-null  object  
 3   GROUP                 542684 non-null  category
 4   SALESRANK             542684 non-null  object  
 5   CATEGORIES            542684 non-null  object  
 6   NUM_SIMILAR           542684 non-null  object  
 7   SIMILAR_PRODUCT_ASIN  379093 non-null  object  
 8   REVIEWS_TOTAL         542684 non-null  object  
 9   REVIEWS_DOWNLOADED    542684 non-null  object  
 10  REVIEWS_AVG_RATING    542684 non-null  object  
dtypes: category(1), object(10)
memory usage: 42.4+ MB


In [197]:
# Save data to expanded columns to csv file 

metadata_df.to_csv('/Users/dirkhartog/Desktop/CUNY_MSDS/DATA_620/Assignment2/amazon_meta_v2.csv', index = False)

In [165]:
## Create a dataframe with only product Id's found in the core subgraph created 
import requests

url = 'https://raw.githubusercontent.com/D-hartog/DATA620/refs/heads/main/Assignment2/amazon0302.txt'  # Replace with the raw URL of your text file
response = requests.get(url)
text_content = response.text

In [166]:
# Open the file
#in_file = csv.reader(open(text_content,'r'))

from_node = []
to_node = []

# Create a graph object
g = net.Graph()

for line in text_content.splitlines():
    line_split = line.split("\t")
    if line_split[0].startswith("#"):
        continue
    g.add_edge(line_split[0],line_split[1], weight = 1)

In [169]:
# Define a function that will trim off the nodes with a degree centrality < 50)
def trim_degrees(go, degree=1):
    go2=go.copy()
    d = dict(net.degree(go2))
    for n in go.nodes():
        if d[n] <= degree:
            #print(n)
            go2.remove_node(n)
    return go2

core100 = trim_degrees(g, degree = 100)

# Number of nodes that we are left with
len(core100.nodes())

77

In [171]:
# largest connected component(LCC)

# Make sure core100 is connected before computing diameter
if not net.is_connected(core100):
    # Work with the largest connected component
    lcc = max(net.connected_components(core100), key=len)
    g_core_lcc = core100.subgraph(lcc).copy()
    print(f"Core100 is NOT fully connected. Using its largest connected component with {len(g_core_lcc.nodes())} nodes.")
else:
    g_core_lcc = core100
    print("Core100 is connected.")

# Compute Diameter :Longest shortest path in the largest connected part of core100.
diameter = net.diameter(g_core_lcc)
print(f"Diameter: {diameter}")

# Compute Average Clustering Coefficient : How tightly nodes form triangles.
avg_clustering = net.average_clustering(core100)
print(f"Average Clustering Coefficient: {avg_clustering:.4f}")

# Compute  Degree Centrality (normalized degree): Nodes with most connections (normalized)
deg_centrality = net.degree_centrality(core100)
top_nodes = sorted(deg_centrality.items(), key=lambda x: -x[1])[:5]
print("Top 5 nodes by degree centrality:")
for node, score in top_nodes:
    print(f"Node {node}: {score:.3f}")

Core100 is NOT fully connected. Using its largest connected component with 20 nodes.
Diameter: 7
Average Clustering Coefficient: 0.0766
Top 5 nodes by degree centrality:
Node 33: 0.066
Node 481: 0.066
Node 4429: 0.066
Node 8: 0.053
Node 2501: 0.053


In [178]:
# Filter metadata on the core nodes
node_list = list(g_core_lcc.nodes())

core_lcc_metadata = metadata_df[metadata_df["ID"].isin(node_list)]

core_lcc_metadata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 6 to 3661
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype   
---  ------                --------------  -----   
 0   ID                    20 non-null     object  
 1   ASIN                  20 non-null     object  
 2   TITLE                 20 non-null     object  
 3   GROUP                 20 non-null     category
 4   SALESRANK             20 non-null     object  
 5   CATEGORIES            20 non-null     object  
 6   NUM_SIMILAR           20 non-null     object  
 7   SIMILAR_PRODUCT_ASIN  11 non-null     object  
 8   REVIEWS_TOTAL         20 non-null     object  
 9   REVIEWS_DOWNLOADED    20 non-null     object  
 10  REVIEWS_AVG_RATING    20 non-null     object  
dtypes: category(1), object(10)
memory usage: 2.1+ KB


In [180]:
core_lcc_metadata.head()

,ID,ASIN,TITLE,GROUP,SALESRANK,CATEGORIES,NUM_SIMILAR,SIMILAR_PRODUCT_ASIN,REVIEWS_TOTAL,REVIEWS_DOWNLOADED,REVIEWS_AVG_RATING
6,6,0486220125,HOW THE OTHER HALF LIVES: STUDIES AMONG THE TE...,BOOK,188784,5,5,0486401960 0452283612 0486229076 0714840343...,17,17,4
8,8,0231118597,LOSING MATT SHEPARD,BOOK,277409,4,5,B000067D0Y 0375727191 080148605X 1560232579...,15,15,4.5
18,18,B000007R0T,SOL TO SOUL,MUSIC,109301,1,5,B000059QC1 B00000JQIE B00029J1X6 B0006TRO68...,15,15,5
33,33,0439240751,"DOUBLE JEOPARDY (T*WITCHES, 6)",BOOK,97166,6,5,0439492327 0439240727 0439240719 0439240700...,4,4,5
34,34,B000002O8D,SOUTHERN BY THE GRACE OF GOD: LYNYRD SKYNYRD T...,MUSIC,89264,5,5,B0000061RJ B000294SBQ B000001Y9Z B000002IRC...,6,6,4


In [ ]:
# Dhanya's code

"""
file_id = '1asJeOfHvL8SjOd5j28SCW4DxzAvX1GvD'
url = f'https://drive.google.com/uc?export=download&id={file_id}'


response = requests.get(url)
meta_text = response.text

print(meta_text[:500])

def parse_metadata(meta_text):
    metadata = {}
    current_id = None
    for line in meta_text.splitlines():
        if line.startswith("Id:"):
            current_id = line.split("Id:")[1].strip()
        elif line.startswith("title:") and current_id:
            metadata[current_id] = line.split("title:")[1].strip()
    return metadata

# Parse metadata
node_metadata = parse_metadata(file)

# Example lookup
print(node_metadata.get("14949"))
""""